In [1]:
# %tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
df = pd.read_csv("adults.csv",index_col=False, sep=", ",engine='python')

In [4]:
df.head

<bound method NDFrame.head of        age         workclass  fnlwgt  education  education-num  \
0       39         State-gov   77516  Bachelors             13   
1       50  Self-emp-not-inc   83311  Bachelors             13   
2       38           Private  215646    HS-grad              9   
3       53           Private  234721       11th              7   
4       28           Private  338409  Bachelors             13   
...    ...               ...     ...        ...            ...   
48837   39           Private  215419  Bachelors             13   
48838   64                 ?  321403    HS-grad              9   
48839   38           Private  374983  Bachelors             13   
48840   44           Private   83891  Bachelors             13   
48841   35      Self-emp-inc  182148  Bachelors             13   

           marital-status         occupation    relationship  \
0           Never-married       Adm-clerical   Not-in-family   
1      Married-civ-spouse    Exec-managerial     

In [5]:
df.shape

(48842, 15)

In [6]:
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
salary            object
dtype: object

In [7]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')

# Preprocessing

### change salary to int 

In [8]:
df.salary.unique()

array(['<=50K', '>50K', '<=50K.', '>50K.'], dtype=object)

In [9]:
df['salary'] = df['salary'].map({'<=50K':0, '>50K':1, '<=50K.':0, '>50K.':1}).astype(int)

### change sex to int

In [10]:
df.sex.unique()

array(['Male', 'Female'], dtype=object)

In [11]:
df['sex'] = df['sex'].map({'Male':1, 'Female':0}).astype(int)

### change ? to np.nan

In [13]:
def count_na(df,ch='?'):
    for i in df.columns:
        if len(df[df[i]==ch]) > 0:
            print(i)
            df[i] = df[i].replace(ch,np.nan)
    return 
count_na(df)

workclass
occupation
native-country


/Users/aab/opt/anaconda3/envs/deep/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


### drop rows with NaN value

In [14]:
df[pd.isnull(df).any(axis=1)].shape

(3620, 15)

In [15]:
df.shape

(48842, 15)

In [16]:
df.dropna(inplace=True)

In [17]:
df.shape

(45222, 15)